
#**BLACK LITTERMAN ASSIGNMENT**

# Installation


In [ ]:
!pip install yfinance

# Importing Packages

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

# Getting Stock Data

In [ ]:
# Define stock tickers, T-bill, and market index tickers
stock_tickers = ['TSLA', 'NFLX', 'NVDA', 'AMD', 'MSFT']
tbill_ticker = '^TNX'  # Risk-free rate
market_ticker = '^GSPC'  # S&P 500 (market benchmark)

# Set date range for one year
end_date = datetime.now()
start_date = end_date - timedelta(days=365)

# Download adjusted close prices for stocks, T-bill, and market index
portfolio = yf.download(stock_tickers, start=start_date, end=end_date)['Close']
Tbill = yf.download(tbill_ticker, start=start_date, end=end_date)[['Close']].rename(columns={'Close': 'tbill'})
market = yf.download(market_ticker, start=start_date, end=end_date)[['Close']].rename(columns={'Close': 'gspc'})

[*********************100%***********************]  5 of 5 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


# Risk Free Interest Rates

In [ ]:
# Remove timezone from indices
portfolio.index = portfolio.index.tz_localize(None)
Tbill.index = Tbill.index.tz_localize(None)
market.index = market.index.tz_localize(None)

# Convert column names to lowercase
portfolio.columns = portfolio.columns.str.lower()

# Check if DataFrames are not empty
if not portfolio.empty and not Tbill.empty and not market.empty:
    # Calculate daily returns
    returns = portfolio.pct_change()
    tbill_rate = Tbill.pct_change()
    market_returns = market.pct_change()

    # Align returns with T-bill rate
    returns, tbill_rate = returns.align(tbill_rate, join='inner', axis=0)
    market_returns, tbill_rate = market_returns.align(tbill_rate, join='inner', axis=0)

    # Subtract T-bill rate from stock returns (excess returns)
    for ticker in ['tsla', 'nflx', 'nvda', 'amd', 'msft']:
        returns[ticker] -= tbill_rate["tbill"]

    # Calculate excess market return
    excess_market_return = pd.DataFrame()
    excess_market_return['gspc'] = market_returns['gspc'] - tbill_rate["tbill"]

    # Drop rows with missing values
    returns.dropna(inplace=True)
    excess_market_return.dropna(inplace=True)

    # Print results
    print("Excess Returns:\n", returns.head())
    print("Excess Market Return:\n", excess_market_return.head())
else:
    print("One or more DataFrames are empty. Check data retrieval.")


Excess Returns:
 Ticker           amd      msft      nflx      nvda      tsla
Date                                                        
2023-09-29  0.005805  0.011948  0.008516  0.014736  0.020806
2023-10-02 -0.019678 -0.004894 -0.016824  0.005441 -0.018539
2023-10-03 -0.056301 -0.051545 -0.034824 -0.053659 -0.045562
2023-10-04  0.053821  0.031726  0.014351  0.025994  0.073296
2023-10-05 -0.007345  0.005056 -0.007634  0.018492 -0.000449
Excess Market Return:
                 gspc
Date                
2023-09-29  0.002511
2023-10-02 -0.023975
2023-10-03 -0.039155
2023-10-04  0.022062
2023-10-05  0.002497


# Price of Risk

\begin{align}
  A = \frac{E(r_m) - r_f}{σ_m^2}
\end{align}

Explanation of Terms:

A: This is the price of risk or the risk aversion coefficient. It measures how much additional return an investor requires for taking on an additional unit of risk.

E(rm
​
 ): This is the expected return of the market portfolio. It represents the average return that an investor expects to earn from the market as a whole.

𝑟
𝑓
​
 : This is the risk-free rate, which is the return on a theoretically riskless investment, such as a government bond.

𝜎
𝑚
2
 : This is the variance of the returns of the market portfolio, which represents the risk or volatility associated with the market's return.

In [ ]:
# Calculate the average excess return of the market
average_market_excess_return = np.mean(excess_market_return['gspc']) # YOUR CODE HERE#

# Calculate the variance of the market returns
market_return_variance = np.var(market_returns['gspc'])# YOUR CODE HERE#

# Compute the price of risk (A)
market_price_of_risk = average_market_excess_return/market_return_variance # YOUR CODE HERE#

# Extract the first value of A
A = market_price_of_risk
A

29.589415618936524

### Variance-Covariance Matrix

A **Variance-Covariance Matrix** is a square matrix where:

- Diagonal elements represent the variances of individual assets.
- Off-diagonal elements represent the covariances between pairs of assets.

For a portfolio with \(n\) assets, the variance-covariance matrix \( \Sigma \) can be written as:

\

\begin{bmatrix}
\sigma_1^2 & \text{Cov}(r_1, r_2) & \cdots & \text{Cov}(r_1, r_n) \\
\text{Cov}(r_2, r_1) & \sigma_2^2 & \cdots & \text{Cov}(r_2, r_n) \\
\vdots & \vdots & \ddots & \vdots \\
\text{Cov}(r_n, r_1) & \text{Cov}(r_n, r_2) & \cdots & \sigma_n^2 \\
\end{bmatrix}


Where:

- \( \sigma_i^2 \) represents the variance of asset \(i\).
- \( \text{Cov}(r_i, r_j) \) represents the covariance between the returns of asset \(i\) and asset \(j\).


In [ ]:
# Calculate the variance-covariance matrix of the asset returns
variance_covariance_matrix = returns.cov() # YOUR CODE HERE#

# Display the matrix
variance_covariance_matrix

Ticker,amd,msft,nflx,nvda,tsla
Ticker,,,,,
amd,0.001171,0.000392,0.000450,0.000818,0.000623
msft,0.000392,0.000379,0.000324,0.000411,0.000404
nflx,0.000450,0.000324,0.000620,0.000481,0.000335
nvda,0.000818,0.000411,0.000481,0.001287,0.000602
tsla,0.000623,0.000404,0.000335,0.000602,0.001535


### Market Weights

**Market Weights** represent the proportion of the portfolio allocated to each asset. For \(n\) assets, the weights are denoted as:

$$
w = \begin{bmatrix} w_1 \\ w_2 \\ \vdots \\ w_n \end{bmatrix}
$$

Where:
- \(w_i\) is the weight of asset \(i\).
- The sum of all weights is 1:
  $$
  \sum_{i=1}^{n} w_i = 1
  $$

### Portfolio Expected Return:
The portfolio's expected return \(E(r_p)\) is the weighted sum of asset returns:

$$
E(r_p) = \sum_{i=1}^{n} w_i E(r_i)
$$

### Portfolio Variance:
The portfolio variance \( \sigma_p^2 \) is calculated as:

$$
\sigma_p^2 = \mathbf{w}^T \Sigma \mathbf{w}
$$

Where:
- \( \Sigma \) is the variance-covariance matrix.


In [ ]:
# Fetch stock tickers for each company
tsla = yf.Ticker('TSLA')
nflx = yf.Ticker('NFLX')
nvda = yf.Ticker('NVDA')
amd = yf.Ticker('AMD')
msft = yf.Ticker('MSFT')

# Retrieve market capitalization for each company, defaulting to 0 if not available
market_capitalizations = {
    'tsla': tsla.info.get('marketCap', 0),
    'nflx': nflx.info.get('marketCap', 0),
    'nvda': nvda.info.get('marketCap', 0),
    'amd': amd.info.get('marketCap', 0),
    'msft': msft.info.get('marketCap', 0)
}

# Calculate the sum of all market capitalizations
total_market_cap = sum(market_capitalizations.values()) # YOUR CODE HERE#

# Initialize an array for the market cap weights
market_cap_weights = np.zeros(len(market_capitalizations))# YOUR CODE HERE#

# Calculate weights based on the proportion of each company's market cap to the total
market_cap_weights[0] = market_capitalizations['tsla'] / total_market_cap # YOUR CODE HERE#
market_cap_weights[1] = market_capitalizations['nflx'] / total_market_cap # YOUR CODE HERE#
market_cap_weights[2] = market_capitalizations['nvda'] / total_market_cap # YOUR CODE HERE#
market_cap_weights[3] = market_capitalizations['amd'] / total_market_cap # YOUR CODE HERE#
market_cap_weights[4] = market_capitalizations['msft'] / total_market_cap # YOUR CODE HERE#

# Print the weights
print("Weights based on market cap: ", market_cap_weights)


Weights based on market cap:  [0.10634038 0.03997818 0.39840601 0.03549464 0.41978078]


### Implied Equilibrium Excess Return

The formula for the **Implied Equilibrium Excess Return** \( \pi \) is given by:

$$
\pi = A \Sigma w
$$


In [ ]:
# Calculate implied equilibrium excess returns using the Black-Litterman model formula
implied_excess_returns = np.dot(np.dot(A,variance_covariance_matrix),market_cap_weights)#YOUR CODE HERE#

# Display the result
implied_excess_returns

array([0.01805043, 0.01094867, 0.01377321, 0.01756098, 0.02609739])

### Views and Link Matrix

(Order: TSLA, NFLX, NVDA, AMD, MSFT)

Let the views and link matrix \( P \) represent the following relationships between these stocks:

$$
P = \begin{bmatrix}
p_{11} & p_{12} & p_{13} & p_{14} & p_{15} \\
p_{21} & p_{22} & p_{23} & p_{24} & p_{25} \\
\vdots & \vdots & \vdots & \vdots & \vdots \\
p_{n1} & p_{n2} & p_{n3} & p_{n4} & p_{n5} \\
\end{bmatrix}
$$

Where:
- $$ p_{ij} $$ represents the view or link between asset \(i\) and asset \(j\).
- The assets are ordered as follows: TSLA, NFLX, NVDA, AMD, MSFT.


## **MY VIEWS**



1.   MSFT outperforms TSLA by 2%
2.   NVDA underperforms AMD by 1.5%
3.   NFLX is expected to have a return of 5%

In [ ]:
# Define the view vector (Q), representing expected excess returns for certain assets
view_expected_returns = [0.02, -0.015, 0.05] #YOUR CODE HERE#

# Define the pick matrix (P), representing the assets involved in the views
view_pick_matrix = np.zeros((3, 5))
view_pick_matrix[0, 0] = -1
view_pick_matrix[0, 4] = 1
view_pick_matrix[1, 2] = -1
view_pick_matrix[1, 3] = 1
view_pick_matrix[2, 1] = 1

view_pick_matrix


array([[-1.,  0.,  0.,  0.,  1.],
       [ 0.,  0., -1.,  1.,  0.],
       [ 0.,  1.,  0.,  0.,  0.]])

##Uncertainty About Views

The formula for the uncertainty about views is given by:

$$
\Omega = \tau P \Sigma P^T
$$

Where:
- $$ \Omega $$: The covariance matrix of the uncertainty about the views.
- $$ \tau $$: A scalar representing the degree of uncertainty in the prior market equilibrium estimate. Commonly set to 1.
- $$ P $$: The view matrix, encoding the investor’s views on different assets.
- $$ \Sigma $$: The variance-covariance matrix of the asset returns.
- $$ P^T $$: The transpose of the view matrix.


In [ ]:
# Calculate the uncertainty (Omega) in the views using the Black-Litterman model
# scalar = 1
view_uncertainty_matrix = np.dot(np.dot(view_pick_matrix,variance_covariance_matrix), np.transpose(view_pick_matrix))#YOUR CODE HERE#

# Display the result
view_uncertainty_matrix

array([[ 1.45954495e-03, -1.00790915e-04,  1.20273809e-05],
       [-1.00790915e-04,  9.45607436e-04,  8.70787645e-05],
       [ 1.20273809e-05,  8.70787645e-05,  3.79131217e-04]])

### Solving first term of Black-Litterman Formula

The formula is:

$$
\left[ (\tau \Sigma)^{-1} + P^T \Omega^{-1} P \right]^{-1}
$$


In [ ]:
# Calculate the first term in the Black-Litterman formula
inverse_adjusted_cov_matrix = np.linalg.inv(np.linalg.inv(variance_covariance_matrix) + np.dot(np.dot(np.transpose(view_pick_matrix), np.linalg.inv(view_uncertainty_matrix)), view_pick_matrix))

# Display the result
inverse_adjusted_cov_matrix

array([[0.00082987, 0.00019602, 0.00028717, 0.00047116, 0.00055613],
       [0.00019602, 0.00018957, 0.00016186, 0.0002054 , 0.00020203],
       [0.00028717, 0.00016186, 0.00044932, 0.0003798 , 0.00023001],
       [0.00047116, 0.0002054 , 0.0003798 , 0.00078308, 0.00036361],
       [0.00055613, 0.00020203, 0.00023001, 0.00036361, 0.00101217]])

### Solving second term of Black-Litterman Formula

The formula is:

$$
\left[ (\tau \Sigma)^{-1} \pi + P^T \Omega^{-1} Q \right]
$$


In [ ]:
# Calculate the second term in the Black-Litterman formula
adjusted_return_term = (
    np.dot(np.linalg.inv(variance_covariance_matrix), implied_excess_returns) +
    np.dot(np.dot(np.transpose(view_pick_matrix), np.linalg.inv(view_uncertainty_matrix)), view_expected_returns)
)
# Display the result
adjusted_return_term

array([ -7.52706815, 139.02224682,  39.20703355, -26.36816671,
        23.09468588])

### Expected Returns Based on Black-Litterman Model

The formula is:

$$
E(r) - r_f = \left[ (\tau \Sigma)^{-1} + P^T \Omega^{-1} P \right]^{-1} \left[ (\tau \Sigma)^{-1} \pi + P^T \Omega^{-1} Q \right]
$$


In [ ]:
# Calculate the expected returns using the Black-Litterman model
bl_expected_returns = np.dot(inverse_adjusted_cov_matrix, adjusted_return_term)

# Display the result
bl_expected_returns

array([0.0326835 , 0.03047433, 0.03325477, 0.02764866, 0.04670698])

# Black Litterman vs Historical Average

In [ ]:
# Print the historical average returns
print("Historical Average Returns: ", returns.mean().to_numpy())

# Print the expected returns from the Black-Litterman model
print("Black-Litterman Expected Returns: ", bl_expected_returns)

Historical Average Returns:  [0.00307194 0.00200833 0.00340467 0.00542239 0.00137226]
Black-Litterman Expected Returns:  [0.0326835  0.03047433 0.03325477 0.02764866 0.04670698]
